In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import StackingRegressor
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings

In [2]:
df = pd.read_csv('Cold_Storage.csv')

In [3]:
df.head()

,Temperature,Humidity,Oxygen,Carbon Dioxide,Nitrogen,Ethylene,Sulfur Dioxide,Moisture Control,Crop
0,4,90,4,5,94,0,0,93,apple
1,3,91,3,3,96,0,0,93,apple
2,1,93,2,3,92,0,0,91,apple
3,4,90,1,5,90,0,0,93,apple
4,3,95,1,5,90,0,0,90,apple


In [4]:
X = df.drop(columns=['Crop'],axis=1)

In [5]:
X.head()

,Temperature,Humidity,Oxygen,Carbon Dioxide,Nitrogen,Ethylene,Sulfur Dioxide,Moisture Control
0,4,90,4,5,94,0,0,93
1,3,91,3,3,96,0,0,93
2,1,93,2,3,92,0,0,91
3,4,90,1,5,90,0,0,93
4,3,95,1,5,90,0,0,90


In [6]:
print("Categories in 'Temperature' variable:     ",end=" " )
print(df['Temperature'].unique())

print("Categories in 'Humidity' variable:  ",end=" ")
print(df['Humidity'].unique())

print("Categories in'Oxygen' variable:",end=" " )
print(df['Oxygen'].unique())

print("Categories in 'Carbon Dioxide' variable:     ",end=" " )
print(df['Carbon Dioxide'].unique())

print("Categories in 'Nitrogen' variable:     ",end=" " )
print(df['Nitrogen'].unique())

print("Categories in 'Ethylene' variable:     ",end=" " )
print(df['Ethylene'].unique())

print("Categories in 'Sulfur Dioxide' variable:     ",end=" " )
print(df['Sulfur Dioxide'].unique())

print("Categories in 'Moisture Control' variable:     ",end=" " )
print(df['Moisture Control'].unique())

Categories in 'Temperature' variable:      [ 4  3  1  2  0 10 14 13 11 12 15 17 16 18 20 19  7  9  5  8  6 28 27 25
 26]
Categories in 'Humidity' variable:   [90 91 93 95 92 94 54 51 58 50 53 56 57 55 59 52 60 15 14 13 12 86 87 88
 85 89 70 62 61 66 63 65 68 67 64 69]
Categories in'Oxygen' variable: [4 3 2 1 5]
Categories in 'Carbon Dioxide' variable:      [ 5  3  2  4 16 12 15 13 14  7  6  8 10  9  1]
Categories in 'Nitrogen' variable:      [94 96 92 90 93 97 91 95 84 82 86 81 80 85 83 79 87 88 89]
Categories in 'Ethylene' variable:      [0]
Categories in 'Sulfur Dioxide' variable:      [0]
Categories in 'Moisture Control' variable:      [93 91 90 95 94 92 13 12 14 15 11 10 87 88 85 89 86 65 62 68 70 63 61 60
 66 67 64 69 56 59 52 55 53 57 50 54 51 58]


In [7]:
y = df['Crop']

In [8]:
y

0          apple
1          apple
2          apple
3          apple
4          apple
           ...  
219995    tomato
219996    tomato
219997    tomato
219998    tomato
219999    tomato
Name: Crop, Length: 220000, dtype: object

In [9]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [10]:
from sklearn.preprocessing import LabelEncoder



# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding
y= label_encoder.fit_transform(y)

# Output the encoded data
print(y)

[ 0  0  0 ... 20 20 20]


In [11]:
X = preprocessor.fit_transform(X)

In [12]:
X.shape

(220000, 8)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((176000, 8), (44000, 8))

In [14]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    'decision_tree': DecisionTreeClassifier(random_state=42),
    'svm': SVR(),
    'neural_network': MLPRegressor(random_state=42, max_iter=1000)
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.6996
- Mean Absolute Error: 4.7080
- R2 Score: 0.1946
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.6501
- Mean Absolute Error: 4.6603
- R2 Score: 0.2001


decision_tree
Model performance for Training set
- Root Mean Squared Error: 3.6005
- Mean Absolute Error: 1.5531
- R2 Score: 0.6786
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4.5026
- Mean Absolute Error: 2.3647
- R2 Score: 0.4920


